In [8]:
from ultralytics import YOLO
import cv2
import numpy as np
import time

In [9]:

class MotionDetector:
    def __init__(self, threshold=150):
        self.threshold = threshold
        self.previous_frame = None
        self.start_time = None
        
    def detect_motion(self, frame):
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        gray = cv2.GaussianBlur(gray, (21, 21), 0)              # Blur the grayscale image to reduce noise
        
        # If this is the first frame, initialize previous frame and return False
        if self.previous_frame is None:
            self.previous_frame = gray
            self.start_time = time.time()
            return False
        
        
        frame_diff = cv2.absdiff(self.previous_frame, gray)
        
        thresholded = cv2.threshold(frame_diff, self.threshold, 255, cv2.THRESH_BINARY)[1]
       
        thresholded = cv2.dilate(thresholded, None, iterations=2)
        
        
        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        if len(contours) > 0:
            
            self.previous_frame = gray             # Motion detected, update previous frame and return True
            self.start_time = time.time()
            return True
        
        self.previous_frame = gray
        return False



md = MotionDetector()


cap = cv2.VideoCapture(0)

start_time = cv2.getTickCount()
countdown_time = 60  
elapsed_time = 0
game_over = False
motiond = 0
end = 0

circle_color = (0, 255, 0)  
circle_radius = 50
circle_thickness = 5
circle_center = (100, 150)
last_color_change_time = time.time()


model = YOLO('yolov8n.pt')  

while True:
    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    
    frame = cv2.resize(frame, (1280, 720))

    
    results = model(frame)

    output_frame = np.zeros_like(frame)

    human_present = False
    
    # Drawing bounding boxes for human detections
    for result in results:
        for box in result.boxes:
           
            if box.cls == 0 and box.conf[0]>0.85:  # YOLOv8 uses '0' for the "person" class in COCO
                human_present = True
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f'Person: {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                
                output_frame[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
    
    if not human_present:
        cv2.putText(frame, "NO HUMAN PRESENT", (frame.shape[1]//2 - 250, frame.shape[0]//2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 12)
        cv2.putText(frame, "NO HUMAN PRESENT", (frame.shape[1]//2 - 250, frame.shape[0]//2), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 8)
        banda_aya = time.time()
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        last_color_change_time = time.time()
        continue

   
    motion_detected = md.detect_motion(output_frame)

    
    current_time = time.time()
    if current_time - last_color_change_time > 3:
        last_color_change_time = current_time
        if circle_color == (0, 255, 0):
            circle_color = (0, 0, 255)  
        else:
            circle_color = (0, 255, 0)  
    
   
    cv2.circle(frame, circle_center, circle_radius, circle_color, circle_thickness)
    
    
    if not game_over:
        
        elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        remaining_time = countdown_time - int(elapsed_time)
        if remaining_time <= 0 or motiond == 1:
            game_over = True
        else:
            cv2.putText(frame, "Time: {}".format(remaining_time), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            if cv2.waitKey(1) & 0xFF == ord('k'):
                
                cv2.putText(frame, "You Win!", (520, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

                win_start_time = time.time()
                while time.time() - win_start_time < 1:
                    cv2.imshow('Frame', frame)
                    cv2.waitKey(1)
                end = 1
                break

    
    if motion_detected:
        if circle_color == (0, 0, 255):
           motiond = 1
        cv2.putText(frame, "Motion Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "Not Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    
    cv2.imshow('Frame', frame)

    if game_over == True and end == 0:
        cv2.putText(frame, "Game Over", (480, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

        game_over_start_time = time.time()
        while time.time() - game_over_start_time < 1:
            cv2.imshow('Frame', frame)
            cv2.waitKey(1)
        end = 1
        break

    
    if cv2.waitKey(1) & 0xFF == ord('q'):          # Exit the loop if 'q' is pressed
        break



cap.release()
cv2.destroyAllWindows()


0: 384x640 2 persons, 123.9ms
Speed: 9.5ms preprocess, 123.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 188.9ms
Speed: 0.0ms preprocess, 188.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 190.0ms
Speed: 0.0ms preprocess, 190.0ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 185.5ms
Speed: 0.0ms preprocess, 185.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 156.6ms
Speed: 3.7ms preprocess, 156.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 132.9ms
Speed: 0.0ms preprocess, 132.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 137.1ms
Speed: 0.0ms preprocess, 137.1ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 136.3ms
Speed: 0.0ms preprocess, 136.3ms inference, 0.0ms postprocess per image 